## Compare acoustic simulations w.r.t. common acoustic parameters

Following Simulations are used:
+ mi3:   Ray Tracing using Mitsuba 3
+ rt:    Ray Tracing using Mitsuba 2 (Leo)
+ ism:   Acoustic Simulation using ISM
+ raven: Acoustic Simulation using raven

In [ ]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from utils import remove_direct
from acoustic_params import EDC, T, C, D50, TS

plt.style.use('ggplot')

### 5x5x5 Box

#### Load Data

In [ ]:
# absorption values used in the compared data
absorpt = [0.1, 0.2, 0.5, 0.7, 0.9]

In [ ]:
mi3_hist = np.load("../data/acoustic-simulations/mi3-box-5-5-5.npy")
mi3_hist = remove_direct(mi3_hist, 3)
mi3_edc  = EDC(mi3_hist)

rt_hist = np.load("../leo/notebooks/data/RT-HIST-box-5-5-5.npy")
rt_hist = remove_direct(rt_hist, 3)
rt_edc  = EDC(rt_hist)

ism_rir = np.load("../leo/notebooks/data/ISM-RIR-5-5-5.npy").T[0]
ism_edc = EDC(ism_rir, energy=False)

In [ ]:
s = 0.0
variant = "Hybrid"
path_root = "../leo/notebooks/raven/box-5-5-5"

times, data = None, None
for a in absorpt:
    path = f"{path_root}/box-*-*-*-a{a:.1f}-s{s:.1f}*/ImpulseResponses/*/*/Histograms/*_{variant}.hst"

    fname = glob.glob(path)[0]
    with open(fname, "r") as file:
        lines = file.read().strip().split('\n')

    while lines[0][:2] != "0=":
        lines.pop(0)

    time, value = [], []
    for l in lines:
        splt = l.strip().split("=")[1].split(",")
        time.append(float(splt[0]))
        value.append(float(splt[2]))

    if data is not None:
        data = np.vstack([data, np.array(value)[None]])
    else:
        times = np.array(time)
        data  = np.array(value)[None]

rav_hist = remove_direct(data.T, 3)[:2000]
rav_time = times[:2000]
rav_edc  = EDC(rav_hist)

In [ ]:
print(mi3_edc.shape)
print(rt_edc.shape)
print(ism_edc.shape)
print(rav_edc.shape)

#### Compare Energy Decay Curves of the simulations for each absorption value

In [ ]:
t0 = np.linspace(0., 2., mi3_edc.shape[0])
t1 = np.linspace(0., 2., rt_edc.shape[0])
t2 = np.linspace(0., 2., ism_edc.shape[0])
t3 = rav_time

plt.figure(figsize=(10, 5))
patch0 = mpatches.Patch(color="C0", label="Mitsuba 3")
patch1 = mpatches.Patch(color="C1", label="Mitsuba 2")
patch2 = mpatches.Patch(color="C2", label="ISM")
patch3 = mpatches.Patch(color="C3", label="RAVEN")
for i, a in enumerate(absorpt):
    if a in [0.1, 0.5, 0.9]:
        plt.plot(t1, rt_edc[:, i],  color="C1")
        plt.plot(t2, ism_edc[:, i], color="C2")
        plt.plot(t3, rav_edc[:, i], color="C3")
        plt.plot(t0, mi3_edc[:, i], color="C0")

plt.ylim(-200., 10.)
plt.xlabel("Time [s]")
plt.ylabel("Energy [dB]")
plt.legend(handles=[patch0, patch1, patch2, patch3])
#plt.savefig("plots/compare-ism-edc.pdf")
plt.show()

#### Compare reverberation time

reverberation time = Nachhaltzeit

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12, 4), sharey=True, layout="constrained")

for i, a in enumerate(absorpt):
    sel_abs = [0.1, 0.5, 0.9]
    if a not in sel_abs:
        continue
    j = sel_abs.index(a)

    mi3_t30 = T(t0, mi3_edc[:, i], dB_init=-10.)
    rt_t30  = T(t1, rt_edc[:, i],  dB_init=-10.)
    ism_t30 = T(t2, ism_edc[:, i], dB_init=-10.)
    rav_t30 = T(t3, rav_edc[:, i], dB_init=-10.)

    axs[j].set_title(f"$\\alpha$ = {a}")

    axs[j].plot(t1, rt_edc[:, i],    alpha=0.8, color='C1', label='Mitsuba 2')
    axs[j].plot(t2, ism_edc[:, i],   alpha=0.8, color='C2', label='ISM')
    axs[j].plot(t3, rav_edc[:, i], alpha=0.8, color='C3', label='raven')
    axs[j].plot(t0, mi3_edc[:, i],   alpha=0.8, color='C0', label='Mitsuba 3')
    axs[j].set_xlim(0., t0[np.argmax(mi3_edc[:, i] < -105.)])

    axs[j].axvline(rt_t30,   linestyle='dashed', color='C1')
    axs[j].axvline(ism_t30,  linestyle='dashed', color='C2')
    # axs[j].axvline(rav_rt30, linestyle='dashed', color='C3')
    axs[j].axvline(mi3_t30,  linestyle='dashed', color='C0')

    axs[j].axhline(-60., linestyle='dotted', color='C4')

    print(f"alpha = {a}, ISM-RT60: {ism_t30:.3f}s, Mi3-RT60: {mi3_t30:.3f}s")

plt.ylim(-100., 5.)
plt.legend()
fig.supxlabel('Time [s]')
fig.supylabel('Energy [dB]')
#plt.savefig('plots/compare-ism-t30.pdf')
plt.show()

#### Compare other acoustic parameters

In [ ]:
mi3_fs = int(mi3_edc.shape[0] / 2.)
rt_fs  = int(rt_edc.shape[0]  / 2.)
ism_fs = int(ism_edc.shape[0] / 2.)
rav_fs = int(rav_edc.shape[0] / 2.)

fig, axs = plt.subplots(2, 2, figsize=(8, 6), sharex=True, layout="constrained")

for i, a in enumerate(absorpt):
    ism_T30 = T(t2, ism_edc[:, i])
    axs[0, 0].scatter(i, ism_T30,              c='C1', marker='1', label='ism')
    axs[0, 0].scatter(i, T(t3, rav_edc[:, i]), c='C2', marker='*', label='raven')
    axs[0, 0].scatter(i, T(t1, rt_edc[:, i]),  c='C4', marker='+', label='mitsuba 2')
    axs[0, 0].scatter(i, T(t0, mi3_edc[:, i]), c='C3', marker='x', label='mitsuba 3')
    axs[0, 0].errorbar(i, ism_T30, yerr=(ism_T30 * 5. / 100.), c='C0', label='JND')

    ism_C80 = C(80., ism_rir[:, i],  ism_fs, energy=False)
    axs[1, 0].scatter(i, ism_C80,                                     c='C1', marker='1')
    axs[1, 0].scatter(i, C(80., rav_hist[:, i], rav_fs, energy=True), c='C2', marker='*')
    axs[1, 0].scatter(i, C(80., rt_hist[:, i],  rt_fs,  energy=True), c='C4', marker='+')
    axs[1, 0].scatter(i, C(80., mi3_hist[:, i], mi3_fs, energy=True), c='C3', marker='x')
    axs[1, 0].errorbar(i, ism_C80, yerr=1., c='C0')

    ism_D50 = D50(ism_rir[:, i],  ism_fs, energy=False)
    axs[0, 1].scatter(i, ism_D50,                                  c='C1', marker='1')
    axs[0, 1].scatter(i, D50(rav_hist[:, i], rav_fs, energy=True), c='C2', marker='*')
    axs[0, 1].scatter(i, D50(rt_hist[:, i],  rt_fs,  energy=True), c='C4', marker='+')
    axs[0, 1].scatter(i, D50(mi3_hist[:, i], mi3_fs, energy=True), c='C3', marker='x')
    axs[0, 1].errorbar(i, ism_D50, yerr=0.05, c='C0')

    ism_TS = TS(t2, ism_rir[:, i])
    axs[1, 1].scatter(i, ism_TS,                 c='C1', marker='1')
    axs[1, 1].scatter(i, TS(t3, rav_hist[:, i]), c='C2', marker='*')
    axs[1, 1].scatter(i, TS(t1, rt_hist[:, i]),  c='C4', marker='+')
    axs[1, 1].scatter(i, TS(t0, mi3_hist[:, i]), c='C3', marker='x')
    axs[1, 1].errorbar(i, ism_TS, yerr=1e-2, c='C0')

    if i == 0:
        axs[0, 0].set_title("$T_{30}$")
        axs[1, 0].set_title("$C_{80}$")
        axs[0, 1].set_title("$D_{50}$")
        axs[1, 1].set_title("$T_{S}$")

        axs[1, 0].set_xlabel("absorption")
        axs[1, 1].set_xlabel("absorption")
        axs[0, 0].set_ylabel("[a.U.]")
        axs[1, 0].set_ylabel("[a.U.]")

        fig.legend(bbox_to_anchor=(1, 0.5), loc='center right')

plt.xticks(range(5), absorpt)
# plt.savefig('plots/compare-params-ism.pdf')
plt.show()

### 25x12x7 Box

#### Load Data

In [ ]:
mi3 = pd.read_pickle("../data/acoustic-simulations/mi3-box-25-12-7.pkl")
# mi2 = pd.read_pickle("../data/mi3-box-25-12-7.pkl")
mi2 = pd.read_pickle("../leo/notebooks/data/mitsuba-box-25-12-7.pkl")
rav = pd.read_pickle("../leo/notebooks/data/raven-box-25-12-7-hybrid.pkl")

mi3_fs, mi2_fs, rav_fs = 1000., 1000., 1000.
mi3_t,  mi2_t,  rav_t  = mi3.index, mi2.index, rav.index
distance_direct = np.linalg.norm(np.array([20., 7., 2.]) - np.array([9., 6., 1.]))

absorption = sorted(list(set(map(lambda x: x.split('-')[0][1:], mi2.columns.to_list()))))
scattering = sorted(list(set(map(lambda x: x.split('-')[1][1:], mi2.columns.to_list()))))

#### Compare Energy Decay Curves

In [ ]:
mi2_ylim_hist = np.max(mi2.max())
mi3_ylim_hist = np.max(mi3.max())
rav_ylim_hist = np.max(rav.max())

for i, s in enumerate(scattering):
    fig, axs = plt.subplots(3, 2, sharex=True, figsize=(12, 9), layout="constrained")
    ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = axs

    ax1.set_ylabel('[a.U.]')
    ax1.set_ylabel('[a.U.]')
    ax2.set_ylabel('[dB]')
    ax3.set_ylabel('[a.U.]')
    ax4.set_ylabel('[dB]')
    ax5.set_ylabel('[a.U.]')
    ax6.set_ylabel('[dB]')
    ax5.set_xlabel('[s]')
    ax6.set_xlabel('[s]')

    for j, a in enumerate(absorption):
        mi3_hist = mi3[f"a{a}-s{s}"]
        mi2_hist = mi2[f"a{a}-s{s}"]
        rav_hist = rav[f"a{a}-s{s}"]

        ax1.plot(mi3_hist, label=f"a{a}-s{s}")
        ax1.set_ylim(mi3_ylim_hist * -0.01, mi3_ylim_hist * 1.01)
        ax1.set_title("Histogram Mi3")
        ax1.legend()

        ax2.plot(mi3_t, EDC(mi3_hist))
        ax2.set_ylim(-105., 5.)
        ax2.set_title("Energy Mi3")

        ax3.plot(mi2_hist)
        ax3.set_ylim(mi2_ylim_hist * -0.01, mi2_ylim_hist * 1.01)
        ax3.set_title("Histogram Mi2")

        ax4.plot(mi2_t, EDC(mi2_hist))
        ax4.set_ylim(-105., 5.)
        ax4.set_title("Energy Mi2")

        ax5.plot(rav_hist)
        ax5.set_ylim(rav_ylim_hist * -0.01, rav_ylim_hist * 1.01)
        ax5.set_title("Histogram Raven")

        ax6.plot(rav_t, EDC(rav_hist))
        ax6.set_ylim(-105., 5.)
        ax6.set_title("Energy Raven")

    plt.show()
    # break

#### Compare EDC + reverberation time

reverberation time = Nachhaltzeit

In [ ]:
for i, s in enumerate(scattering):
    fig, axs = plt.subplots(1, 5, figsize=(20, 5), sharey=True, layout="constrained")
    for j, a in enumerate(absorption):
        mi3_edc = EDC(mi3[f"a{a}-s{s}"])
        mi2_edc = EDC(mi2[f"a{a}-s{s}"])
        rav_edc = EDC(rav[f"a{a}-s{s}"])

        mi3_t30 = T(mi3_t, mi3_edc)
        mi2_t30 = T(mi2_t, mi2_edc)
        rav_t30 = T(rav_t, rav_edc)

        axs[j].plot(mi2_t, mi2_edc, 'C1', label="mi2")
        axs[j].plot(rav_t, rav_edc, 'C2', label="raven")
        axs[j].plot(mi3_t, mi3_edc, 'C0', label="mi3")

        axs[j].axvline(mi3_t30, linestyle='dashed', color='C1', label='T30')
        axs[j].axvline(mi2_t30, linestyle='dashed', color='C2')
        axs[j].axvline(rav_t30, linestyle='dashed', color='C0')

        axs[j].axhline(-60., linestyle='dotted', color='C3')

        axs[j].set_xlim(-0.1, np.max([mi3_t[np.argmax(mi3_edc < -100.)], mi3_t30, mi2_t30, rav_t30]) * 1.05)
        axs[j].set_ylim(-105., 5.)

        axs[j].set_xlabel("Time [s]")
        axs[j].set_ylabel("Energy [dB]")

        axs[j].set_title(f"s = {s} - a = {a}")

    plt.legend()
    #plt.savefig(f'plots/compare-raven-edc-s{s}.pdf')
    plt.show()

#### Compare reverberation time

In [ ]:
mi3_t30 = mi3.apply(lambda x: T(mi3_t, EDC(x))).sort_index()
mi2_t30 = mi2.apply(lambda x: T(mi2_t, EDC(x))).sort_index()
rav_t30 = rav.apply(lambda x: T(rav_t, EDC(x))).sort_index()

df_t30 = pd.DataFrame()

df_t30['mi3'] = mi3_t30
df_t30['mi2'] = mi2_t30
df_t30['rav'] = rav_t30
df_t30['Mi3 T_30 - deviation to Raven [%]'] = 100. * (mi3_t30 -  rav_t30) / rav_t30
df_t30['Mi2 T_30 - deviation to Raven [%]'] = 100. * (mi2_t30 -  rav_t30) / rav_t30

display(df_t30)

In [ ]:
mi3_t10 = mi3.apply(lambda x: T(mi3_t, EDC(x), dB_init=-1., dB_decay=10.)).sort_index()
mi2_t10 = mi2.apply(lambda x: T(mi2_t, EDC(x), dB_init=-1., dB_decay=10.)).sort_index()
rav_t10 = rav.apply(lambda x: T(rav_t, EDC(x), dB_init=-1., dB_decay=10.)).sort_index()

df_t10 = pd.DataFrame()

df_t10['mi3'] = mi3_t10
df_t10['mi2'] = mi2_t10
df_t10['rav'] = rav_t10
df_t10['Mi3 T_10 - deviation to Raven [%]'] = 100. * (mi3_t10 -  rav_t10) / rav_t10
df_t10['Mi2 T_10 - deviation to Raven [%]'] = 100. * (mi2_t10 -  rav_t10) / rav_t10

display(df_t10)

In [ ]:
plt.figure(figsize=(6, 4))
for i, s in enumerate(scattering):
    for j, a in enumerate(absorption):
        idx = f"a{a}-s{s}"
        plt.scatter(s, rav_t30[idx], color='C1', alpha=float(a), label="Mi3")
        plt.scatter(s, mi3_t30[idx], color='C0', alpha=float(a), label="Raven")
        if i == j == 0:
            plt.legend()

plt.xlabel('scattering')
plt.ylabel("$T_{30}$ [s]")
plt.show()

In [ ]:
mi3_deviation = np.zeros((len(scattering), len(absorption)))
mi2_deviation = np.zeros((len(scattering), len(absorption)))

for i, s in enumerate(scattering):
    for j, a in enumerate(absorption):
        idx = f"a{a}-s{s}"
        mi3_deviation[i, j] = df_t30.loc[idx, "Mi3 T_30 - deviation to Raven [%]"]
        mi2_deviation[i, j] = df_t30.loc[idx, "Mi2 T_30 - deviation to Raven [%]"]

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, layout="constrained")

ax1.imshow(np.abs(mi2_deviation), origin='lower', interpolation='none', vmin=0., vmax=7)
ax1.set_title('Mi2 T30 deviation')
ax1.set_xlabel("absorption")
ax1.set_ylabel("scattering")
ax1.set_xticks(range(len(absorption)), absorption)
ax1.set_yticks(range(len(scattering)), scattering)
ax1.grid(False)

ax2img = ax2.imshow(np.abs(mi3_deviation), origin='lower', interpolation='none', vmin=0., vmax=7)
ax2.set_title('Mi3 T30 deviation')
ax2.set_xlabel("absorption")
ax2.set_xticks(range(len(absorption)), absorption)
ax2.grid(False)

fig.colorbar(ax2img, fraction=0.05, pad=0.05).set_label('deviation [%]')

# plt.savefig('./plots/t30dev.pdf')
plt.show()

#### Compare C80

In [ ]:
mi3_c80 = mi3.apply(lambda x: C(80., x, mi3_fs)).sort_index()
mi2_c80 = mi2.apply(lambda x: C(80., x, mi2_fs)).sort_index()
rav_c80 = rav.apply(lambda x: C(80., x, rav_fs)).sort_index()

mi3_deviation = np.zeros((len(scattering), len(absorption)))
mi2_deviation = np.zeros((len(scattering), len(absorption)))

for i, s in enumerate(scattering):
    for j, a in enumerate(absorption):
        idx = f"a{a}-s{s}"
        mi3_deviation[i, j] = mi3_c80[idx] - rav_c80[idx]
        mi2_deviation[i, j] = mi2_c80[idx] - rav_c80[idx]

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, layout="constrained")

ax1.imshow(np.abs(mi2_deviation), origin='lower', interpolation='none', vmin=0., vmax=1.5)
ax1.set_title('Mi2 C80 deviation')
ax1.set_xlabel("absorption")
ax1.set_ylabel("scattering")
ax1.set_xticks(range(len(absorption)), absorption)
ax1.set_yticks(range(len(scattering)), scattering)
ax1.grid(False)

ax2img = ax2.imshow(np.abs(mi3_deviation), origin='lower', interpolation='none', vmin=0., vmax=1.5)
ax2.set_title('Mi3 C80 deviation')
ax2.set_xlabel("absorption")
ax2.set_xticks(range(len(absorption)), absorption)
ax2.grid(False)

fig.colorbar(ax2img, fraction=0.05, pad=0.05).set_label('deviation [%]')

# plt.savefig('./plots/c80dev.pdf')
plt.show()

#### Compare D50

In [ ]:
mi3_d50 = mi3.apply(lambda x: D50(x, mi3_fs)).sort_index()
mi2_d50 = mi2.apply(lambda x: D50(x, mi2_fs)).sort_index()
rav_d50 = rav.apply(lambda x: D50(x, rav_fs)).sort_index()

mi3_deviation = np.zeros((len(scattering), len(absorption)))
mi2_deviation = np.zeros((len(scattering), len(absorption)))

for i, s in enumerate(scattering):
    for j, a in enumerate(absorption):
        idx = f"a{a}-s{s}"
        mi3_deviation[i, j] = mi3_d50[idx] - rav_d50[idx]
        mi2_deviation[i, j] = mi2_d50[idx] - rav_d50[idx]

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, layout="constrained")

ax1.imshow(np.abs(mi2_deviation), origin='lower', interpolation='none', vmin=0., vmax=0.05)
ax1.set_title('Mi2 D50 deviation')
ax1.set_xlabel("absorption")
ax1.set_ylabel("scattering")
ax1.set_xticks(range(len(absorption)), absorption)
ax1.set_yticks(range(len(scattering)), scattering)
ax1.grid(False)

ax2img = ax2.imshow(np.abs(mi3_deviation), origin='lower', interpolation='none', vmin=0., vmax=0.05)
ax2.set_title('Mi3 D50 deviation')
ax2.set_xlabel("absorption")
ax2.set_xticks(range(len(absorption)), absorption)
ax2.grid(False)

fig.colorbar(ax2img, fraction=0.05, pad=0.05).set_label('deviation [%]')

# plt.savefig('./plots/d50dev.pdf')
plt.show()

#### Compare TS

In [ ]:
mi3_ts = mi3.apply(lambda x: TS(mi3_t, x)).sort_index()
mi2_ts = mi2.apply(lambda x: TS(mi2_t, x)).sort_index()
rav_ts = rav.apply(lambda x: TS(rav_t, x)).sort_index()

mi3_deviation = np.zeros((len(scattering), len(absorption)))
mi2_deviation = np.zeros((len(scattering), len(absorption)))

for i, s in enumerate(scattering):
    for j, a in enumerate(absorption):
        idx = f"a{a}-s{s}"
        mi3_deviation[i, j] = mi3_ts[idx] - rav_ts[idx]
        mi2_deviation[i, j] = mi2_ts[idx] - rav_ts[idx]

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, layout="constrained")

ax1.imshow(np.abs(mi2_deviation * 100.), origin='lower', interpolation='none', vmin=0., vmax=1.)
ax1.set_title('Mi2 TS deviation')
ax1.set_xlabel("absorption")
ax1.set_ylabel("scattering")
ax1.set_xticks(range(len(absorption)), absorption)
ax1.set_yticks(range(len(scattering)), scattering)
ax1.grid(False)

ax2img = ax2.imshow(np.abs(mi3_deviation * 100.), origin='lower', interpolation='none', vmin=0., vmax=1.)
ax2.set_title('Mi3 TS deviation')
ax2.set_xlabel("absorption")
ax2.set_xticks(range(len(absorption)), absorption)
ax2.grid(False)

fig.colorbar(ax2img, fraction=0.05, pad=0.05).set_label('deviation [%]')

# plt.savefig('./plots/tsdev.pdf')
plt.show()

#### Visualize T30 linear fit

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(15, 10), sharey=True, layout="constrained")

for j, (s, a) in enumerate([(0.5, 0.5), (0.5, 0.8), (0.5, 0.9), (0.8, 0.8)]):
    x = j // 2
    y = j % 2

    mi3_edc = EDC(mi3[f"a{a}-s{s}"])
    mi2_edc = EDC(mi2[f"a{a}-s{s}"])
    rav_edc = EDC(rav[f"a{a}-s{s}"])

    axs[y, x].plot(rav_t, rav_edc, c='C2', label='Raven')
    axs[y, x].plot(mi2_t, mi2_edc, c='C1', label='Mi2')
    axs[y, x].plot(mi3_t, mi3_edc, c='C0', label='Mi3')

    rav_t30 = T(rav_t, rav_edc)
    axs[y, x].axvline(rav_t30,           color='C2', linestyle='dashed', label='T30 Raven')
    axs[y, x].axvline(T(mi2_t, mi2_edc), color='C1', linestyle='dashed', label='T30 Mi2')
    axs[y, x].axvline(T(mi3_t, mi3_edc), color='C0', linestyle='dashed', label='T30 Mi3')

    axs[y, x].axhline( -5., c='C4',  linestyle='dotted')
    axs[y, x].axhline(-35., c='C4',  linestyle='dotted', label='$T_{30}$ linear fit region')
    axs[y, x].axhline(-60., c='C5',  linestyle='dashed', label='-60dB')

    axs[y, x].errorbar(rav_t30, -20., xerr=0.05 * rav_t30, color='C3', label='JND')

    axs[y, x].set_xlim()
    axs[y, x].set_ylim(-65., 5.)
    axs[y, x].set_xlim(-0.01, mi3_t[np.argmax(mi3_edc < -65.)])

    axs[y, x].set_title(f"s = {s} - a = {a}")

fig.supylabel('Energy [dB]')
fig.supxlabel('Time [s]')
axs[1, 0].legend()
#plt.savefig('plots/compare-raven-errs.pdf')
plt.show()